## Apply all Cell-Health Models to Training and Testing Sets

**Gregory Way, 2019**

In [1]:
import os
import pandas as pd
from joblib import load

from scripts.ml_utils import load_train_test, load_models

In [2]:
%matplotlib inline

In [3]:
def apply_model(model, feature, train_x, test_x):
    """
    Apply model to training and testing matrix
    """
    pred_train_df = (
        pd.DataFrame(model.predict(train_x), columns=["score"])
        .assign(profiles=train_x.index,
                Metadata_data_type="train",
                model=feature)
    )
    pred_test_df = (
        pd.DataFrame(model.predict(test_x), columns=["score"])
        .assign(profiles=test_x.index,
                Metadata_data_type="test",
                model=feature)
    )

    pred_df = pd.concat([pred_train_df, pred_test_df]).reset_index(drop=True)
    return pred_df

def sample_squared_error(scores, y):
    """
    Calculate the squared error per sample depending on model scores
    """
    metadata_cols = [x for x in scores.columns if x.startswith("Metadata_")]
    scores_values = scores.drop(metadata_cols, axis="columns")
    
    all_squared_error = {}
    for cell_health_feature in scores_values.columns:
        y_subset_df = y.loc[:, cell_health_feature].dropna().T
        scores_subset = scores_values.loc[:, cell_health_feature].reindex(y_subset_df.index).T

        squared_error = (y_subset_df - scores_subset) ** 2
        all_squared_error[cell_health_feature] = squared_error
    
    return pd.DataFrame(all_squared_error).reindex(scores.index)

## 1) Load Models and Model Coefficients

For real data and shuffled model data.

In [4]:
consensus = "modz"

In [5]:
model_dict, model_coef = load_models(consensus=consensus)
shuffle_model_dict, shuffle_model_coef = load_models(shuffle=True, consensus=consensus)

In [6]:
# Load Metadata Mapping File
data_dir = os.path.join("..", "1.generate-profiles", "data")
file = os.path.join(data_dir, "profile_id_metadata_mapping.tsv")
metadata_df = pd.read_csv(file, sep='\t')

metadata_df.head()

,Metadata_profile_id,Metadata_cell_line,Metadata_pert_name
0,profile_0,A549,AKT1-1
1,profile_1,A549,AKT1-2
2,profile_2,A549,ARID1B-1
3,profile_3,A549,ARID1B-2
4,profile_4,A549,ATF4-1


## 2) Load Training and Testing Data

In [7]:
x_train_df, x_test_df, y_train_df, y_test_df = load_train_test(drop_metadata=True, consensus=consensus)

## 3) Output Model Coefficients

In [8]:
# Extract all model coefficients and output to file
coef_df = pd.DataFrame(model_coef)
coef_df.index = x_test_df.columns
coef_df.index.name = "features"

file = os.path.join("results",
                    "all_model_coefficients_{}.tsv".format(consensus))
coef_df.to_csv(file, sep='\t', index=True)

print(coef_df.shape)
coef_df.head(2)

(1598, 70)


,cell_health_modz_target_cc_mitosis_ph3_pos_high_n_spots_h2ax_mean,cell_health_modz_target_cc_g2_ph3_pos_n_spots_per_nucleus_area_mean,cell_health_modz_target_cc_all_nucleus_roundness_mean,cell_health_modz_target_cc_polyploid_n_objects,cell_health_modz_target_cc_g2_ph3_neg_n_spots_per_nucleus_area_mean,cell_health_modz_target_cc_edu_pos_alexa647_intensity_nucleus_area_sum,cell_health_modz_target_cc_g1_n_objects,cell_health_modz_target_vb_num_live_cells,cell_health_modz_target_vb_percent_live,cell_health_modz_target_cc_cc_n_spots_mean,...,cell_health_modz_target_cc_all_high_n_spots_h2ax_mean,cell_health_modz_target_cc_all_n_spots_mean,cell_health_modz_target_cc_g2_ph3_neg_high_n_spots_h2ax_mean,cell_health_modz_target_cc_cc_high_n_spots_h2ax_mean,cell_health_modz_target_cc_g1_high_n_spots_h2ax_mean,cell_health_modz_target_cc_polynuclear_high_n_spots_h2ax_mean,cell_health_modz_target_vb_percent_dead_only,cell_health_modz_target_cc_g2_ph3_neg_n_objects,cell_health_modz_target_cc_cc_ph3_pos_hoechst_mitosis_mean,cell_health_modz_target_vb_live_cell_width_length
features,,,,,,,,,,,,,,,,,,,,,
Cells_AreaShape_Area,-0.0,0.0,0.0,-0.0,-0.0,0.000000,-0.0,-0.0,0.0,-0.0,...,-0.0,-0.000000,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0
Cells_AreaShape_Center_X,-0.0,-0.0,-0.0,0.0,0.0,0.024515,0.0,0.0,0.0,-0.0,...,-0.0,-0.002888,0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0


In [9]:
# Extract all model coefficients and output to file
shuffle_coef_df = pd.DataFrame(shuffle_model_coef)
shuffle_coef_df.index = x_test_df.columns
shuffle_coef_df.index.name = "features"

file = os.path.join("results",
                    "all_model_coefficients_shuffled_{}.tsv".format(consensus))
shuffle_coef_df.to_csv(file, sep='\t', index=True)

print(shuffle_coef_df.shape)
shuffle_coef_df.head(2)

(1598, 70)


,cell_health_modz_target_cc_all_n_objects,cell_health_modz_target_vb_num_live_cells,cell_health_modz_target_cc_g1_n_objects,cell_health_modz_target_cc_mitosis_ph3_neg_high_n_spots_h2ax_mean,cell_health_modz_target_cc_g2_ph3_pos_n_objects,cell_health_modz_target_cc_g2_ph3_neg_high_n_spots_h2ax_mean,cell_health_modz_target_cc_polyploid_n_objects,cell_health_modz_target_vb_infection_percentage,cell_health_modz_target_vb_live_cell_roundness,cell_health_modz_target_cc_edu_pos_high_n_spots_h2ax_mean,...,cell_health_modz_target_cc_mitosis_ph3_pos_n_objects,cell_health_modz_target_cc_g2_plus_all_m,cell_health_modz_target_cc_g2_ph3_neg_n_objects,cell_health_modz_target_cc_g1_n_spots_per_nucleus_area_mean,cell_health_modz_target_cc_polynuclear_n_objects,cell_health_modz_target_cc_edu_pos_n_spots_per_nucleus_area_mean,cell_health_modz_target_cc_all_high_n_spots_h2ax_mean,cell_health_modz_target_cc_cc_g2_ph3_pos_early_mitosis_mean,cell_health_modz_target_cc_polyploid_high_n_spots_h2ax_mean,cell_health_modz_target_cc_all_n_spots_mean
features,,,,,,,,,,,,,,,,,,,,,
Cells_AreaShape_Area,-0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,0.0,...,-0.0,-0.0,-0.0,0.030886,-0.0,-0.0,-0.0,0.0,-0.0,-0.0
Cells_AreaShape_Center_X,-0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0,0.0,-0.0,...,-0.0,0.0,-0.0,0.000000,0.0,0.0,0.0,0.0,0.0,-0.0


## 4) Apply all models

For real and shuffled data.

In [10]:
all_scores = []
all_shuffle_scores = []
for cell_health_feature in model_dict.keys():
    # Apply Real Model Classifiers
    model_clf = model_dict[cell_health_feature]
    pred_df = apply_model(model=model_clf,
                          feature=cell_health_feature,
                          train_x=x_train_df,
                          test_x=x_test_df)
    all_scores.append(pred_df)
    
    # Apply Shuffled Model Classifiers
    shuffle_model_clf = shuffle_model_dict[cell_health_feature]
    shuffle_pred_df = apply_model(model=shuffle_model_clf,
                                  feature=cell_health_feature,
                                  train_x=x_train_df,
                                  test_x=x_test_df)
    all_shuffle_scores.append(shuffle_pred_df)

## 5) Concatenate scores with Metadata

In [11]:
# Concatenate real data scores
all_scores = (
    pd.concat(all_scores)
    .reset_index(drop=True)
    .pivot_table(index=["profiles", "Metadata_data_type"],
                 columns="model",
                 values="score")
    .reset_index()
)

all_scores = (
    metadata_df.merge(all_scores,
                      left_on="Metadata_profile_id",
                      right_on="profiles")
    .drop("profiles", axis="columns")
)

all_scores.index = all_scores.Metadata_profile_id
all_scores = all_scores.drop("Metadata_profile_id", axis="columns")

# Remove prefix of variable columns
strip_text = "cell_health_{}_target_".format(consensus)
all_scores.columns = [x.replace(strip_text, "") for x in all_scores.columns]

# Output file
file = os.path.join("results",
                    "all_model_predictions_{}.tsv".format(consensus))
all_scores.to_csv(file, sep='\t', index=True)

print(all_scores.shape)
all_scores.head(2)

(357, 73)


,Metadata_cell_line,Metadata_pert_name,Metadata_data_type,cc_all_high_n_spots_h2ax_mean,cc_all_large_notround_polynuclear_mean,cc_all_large_round_polyploid_mean,cc_all_n_objects,cc_all_n_spots_mean,cc_all_n_spots_per_nucleus_area_mean,cc_all_nucleus_area_mean,...,vb_num_live_cells,vb_percent_all_apoptosis,vb_percent_all_early_apoptosis,vb_percent_all_late_apoptosis,vb_percent_caspase_dead_only,vb_percent_dead,vb_percent_dead_only,vb_percent_live,vb_ros_back_mean,vb_ros_mean
Metadata_profile_id,,,,,,,,,,,,,,,,,,,,,
profile_0,A549,AKT1-1,train,0.287385,0.481496,0.476958,0.012987,0.365384,0.285530,-0.053421,...,0.051556,0.034797,-0.017588,-0.093624,0.280256,-0.123296,0.099040,0.123065,0.281025,0.304451
profile_1,A549,AKT1-2,train,0.112669,0.422407,-0.068795,0.243455,0.061869,0.052564,-0.258505,...,0.185325,0.116329,0.092636,0.176276,0.259545,0.193615,0.276687,-0.193720,0.322336,0.397855


In [12]:
# Concatenate shuffled data scores
all_shuffle_scores = (
    pd.concat(all_shuffle_scores)
    .reset_index(drop=True)
    .pivot_table(index=["profiles", "Metadata_data_type"],
                 columns="model",
                 values="score")
    .reset_index()
)

all_shuffle_scores = (
    metadata_df.merge(all_shuffle_scores,
                      left_on="Metadata_profile_id",
                      right_on="profiles")
    .drop("profiles", axis="columns")
)

all_shuffle_scores.index = all_shuffle_scores.Metadata_profile_id
all_shuffle_scores = all_shuffle_scores.drop("Metadata_profile_id", axis="columns")

# Remove prefix of variable columns
strip_text = "cell_health_{}_target_".format(consensus)
all_shuffle_scores.columns = [x.replace(strip_text, "") for x in all_shuffle_scores.columns]

# Output file
file = os.path.join("results",
                    "all_model_predictions_shuffled_{}.tsv".format(consensus))
all_shuffle_scores.to_csv(file, sep='\t', index=True)

print(all_shuffle_scores.shape)
all_shuffle_scores.head(2)

(357, 73)


,Metadata_cell_line,Metadata_pert_name,Metadata_data_type,cc_all_high_n_spots_h2ax_mean,cc_all_large_notround_polynuclear_mean,cc_all_large_round_polyploid_mean,cc_all_n_objects,cc_all_n_spots_mean,cc_all_n_spots_per_nucleus_area_mean,cc_all_nucleus_area_mean,...,vb_num_live_cells,vb_percent_all_apoptosis,vb_percent_all_early_apoptosis,vb_percent_all_late_apoptosis,vb_percent_caspase_dead_only,vb_percent_dead,vb_percent_dead_only,vb_percent_live,vb_ros_back_mean,vb_ros_mean
Metadata_profile_id,,,,,,,,,,,,,,,,,,,,,
profile_0,A549,AKT1-1,train,0.268431,0.255810,0.154263,-0.161205,0.271800,0.269973,0.267271,...,-0.112639,0.204490,0.152074,0.079872,0.214405,0.208593,0.147448,-0.140079,0.088978,0.024817
profile_1,A549,AKT1-2,train,0.246190,0.288148,0.106719,-0.120006,0.258706,0.274480,0.226279,...,-0.074135,0.156508,0.151970,0.149402,0.214405,0.149165,0.159970,-0.132025,0.125525,0.029538


## 6) Calculate the Squared Error of Individual Samples

For real and shuffled data

In [13]:
y_df = pd.concat([y_train_df, y_test_df]).reindex(all_scores.index)

print(y_df.shape)
y_df.head(2)

(357, 70)


,cc_all_high_n_spots_h2ax_mean,cc_all_large_notround_polynuclear_mean,cc_all_large_round_polyploid_mean,cc_all_n_objects,cc_all_n_spots_mean,cc_all_n_spots_per_nucleus_area_mean,cc_all_nucleus_area_mean,cc_all_nucleus_roundness_mean,cc_cc_edu_pos_mean,cc_cc_g1_mean,...,vb_num_live_cells,vb_percent_all_apoptosis,vb_percent_all_early_apoptosis,vb_percent_all_late_apoptosis,vb_percent_caspase_dead_only,vb_percent_dead,vb_percent_dead_only,vb_percent_live,vb_ros_back_mean,vb_ros_mean
Metadata_profile_id,,,,,,,,,,,,,,,,,,,,,
profile_0,-0.005795,0.580351,0.013975,0.381958,0.150696,0.162511,-0.167603,0.040322,0.091643,0.043915,...,0.438339,0.059414,0.082424,0.000000,-0.065050,-0.020236,-0.007970,0.020263,0.408214,0.654575
profile_1,0.050169,1.277730,0.241808,0.577422,0.220829,0.366989,-0.278044,-0.182571,0.893453,-1.023968,...,0.067568,0.256141,0.132834,0.386327,0.575026,0.225091,0.220461,-0.224965,0.284962,0.567898


In [14]:
all_score_error = sample_squared_error(scores=all_scores, y=y_df)

all_score_error = (
    metadata_df.merge(all_score_error,
                      left_on="Metadata_profile_id",
                      right_index=True)
)

# Output file
file = os.path.join("results",
                    "all_model_sample_squared_error_{}.tsv".format(consensus))
all_score_error.to_csv(file, sep='\t', index=False)

print(all_score_error.shape)
all_score_error.head(2)

(357, 73)


,Metadata_profile_id,Metadata_cell_line,Metadata_pert_name,cc_all_high_n_spots_h2ax_mean,cc_all_large_notround_polynuclear_mean,cc_all_large_round_polyploid_mean,cc_all_n_objects,cc_all_n_spots_mean,cc_all_n_spots_per_nucleus_area_mean,cc_all_nucleus_area_mean,...,vb_num_live_cells,vb_percent_all_apoptosis,vb_percent_all_early_apoptosis,vb_percent_all_late_apoptosis,vb_percent_caspase_dead_only,vb_percent_dead,vb_percent_dead_only,vb_percent_live,vb_ros_back_mean,vb_ros_mean
0,profile_0,A549,AKT1-1,0.085954,0.009772,0.214353,0.136140,0.046091,0.015134,0.013038,...,0.149601,0.000606,0.010002,0.008765,0.119236,0.010621,0.011451,0.010568,0.016177,0.122587
1,profile_1,A549,AKT1-2,0.003906,0.731579,0.096475,0.111534,0.025268,0.098863,0.000382,...,0.013867,0.019547,0.001616,0.044122,0.099528,0.000991,0.003161,0.000976,0.001397,0.028915


In [15]:
all_shuffle_score_error = sample_squared_error(scores=all_shuffle_scores, y=y_df)

all_shuffle_score_error = (
    metadata_df.merge(all_shuffle_score_error,
                      left_on="Metadata_profile_id",
                      right_index=True)
)

# Output file
file = os.path.join("results",
                    "all_model_sample_squared_error_shuffled_{}.tsv".format(consensus))
all_shuffle_score_error.to_csv(file, sep='\t', index=False)

print(all_shuffle_score_error.shape)
all_shuffle_score_error.head()

(357, 73)


,Metadata_profile_id,Metadata_cell_line,Metadata_pert_name,cc_all_high_n_spots_h2ax_mean,cc_all_large_notround_polynuclear_mean,cc_all_large_round_polyploid_mean,cc_all_n_objects,cc_all_n_spots_mean,cc_all_n_spots_per_nucleus_area_mean,cc_all_nucleus_area_mean,...,vb_num_live_cells,vb_percent_all_apoptosis,vb_percent_all_early_apoptosis,vb_percent_all_late_apoptosis,vb_percent_caspase_dead_only,vb_percent_dead,vb_percent_dead_only,vb_percent_live,vb_ros_back_mean,vb_ros_mean
0,profile_0,A549,AKT1-1,0.075200,0.105327,0.019681,0.295026,0.014666,0.011548,0.189115,...,0.303577,0.021047,0.004851,0.006379,0.078095,0.052363,0.024155,0.025710,0.101912,0.396595
1,profile_1,A549,AKT1-2,0.038424,0.979272,0.018249,0.486405,0.001435,0.008558,0.254342,...,0.020080,0.009927,0.000366,0.056134,0.130047,0.005765,0.003659,0.008638,0.025420,0.289832
2,profile_2,A549,ARID1B-1,0.001568,0.830465,0.002182,0.198953,0.008911,0.059113,0.292325,...,0.014765,0.022593,0.009508,0.038848,0.082677,0.010804,0.004089,0.003641,0.077875,0.057660
3,profile_3,A549,ARID1B-2,0.022816,0.002304,0.054945,0.082557,0.467998,0.324034,0.114493,...,0.137152,0.012839,0.000246,0.000840,0.078108,0.114720,0.028576,0.036902,0.092714,0.033436
4,profile_4,A549,ATF4-1,21.969057,0.058514,6.558383,4.181768,6.080282,7.932581,5.400816,...,5.075948,0.084586,0.032120,0.413246,0.086914,0.720322,0.147279,0.000015,0.436144,0.011196
